In [1]:
import tensorflow as tf
import numpy as np
import warnings; 
warnings.simplefilter('ignore')

In [3]:
#Reading the minist data
#setting one_hot=True means i want y labels of my data to be one hot encoded
#ex->suppose i have y which takes 3 values 1,2,3
#one hot encoded will be [1,0,0],[0,1,0],[0,0,1]
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
#As i discussed previously if we do not know the shape of data we will use placeholder 
#our input is 28*28 pixel image of handwritten digits which are in grey scale(means black and white picture)
#so we flatten the 28*28 to 784 dimension 
#Here x is the input to the neural network
x = tf.placeholder(tf.float32, [None, 784])
# now declare the output data placeholder - 10 digits [0,1,2,3,4,5,6,7,8,9]
# y is the ouput value
y = tf.placeholder(tf.float32, [None, 10])

In [5]:
# I want my architecture is to one hidden layer 
# so my architecture is 784(input)---->300(hidden layer)---->10(output)
# name parameter basically means what name we are giving to that variable(which is optional)
# So for the input and hidden layer one weight matrix will be there which will be of shape (784,300)
# So i initialized W1 matrix which takes values from normal dist with std-dev 0.03
# I created this matrix using tf.variable because all the values in this matrix will change it is not constant
W1 = tf.Variable(tf.random_normal([784, 300], stddev=0.03),name='W1')
# For that weight matrix baises term will be there which will take 300 values
b1 = tf.Variable(tf.random_normal([300]), name='b1')
# similarly for hidden layer and output layer
# the weight matrix W2 will have shape 300,10 
W2 = tf.Variable(tf.random_normal([300,10], stddev=0.03),name='W2')
# Same for biases term also 
b2 = tf.Variable(tf.random_normal([10]), name='b2')

In [6]:
# Now for the forward propagation
# first input values will be multiply with the W1 matrix 
# Then biases term will be added
# After that ReLu non linearity is added
hidden_out = tf.add(tf.matmul(x, W1), b1)
hidden_out = tf.nn.relu(hidden_out)

In [7]:
# Here the value of hidden_out goes to the W2 Matrix and we multiply with it and added the biases
# Then applying softmax to it then storing it to y_
y_hidden_out = tf.add(tf.matmul(hidden_out, W2), b2)
y_ = tf.nn.softmax(y_hidden_out)
# What softmax is bascially doing?
# this  tf.add(tf.matmul(hidden_out, W2), b2) returns 10 values as output because we have 10 nodes at output
# applying softmax on top of it normalizes that 10 values so that the 10 values added upto 1
# Now that 10 values can act as probability score for each class

In [8]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_hidden_out, labels=y))

W1111 09:40:19.521057 10028 deprecation.py:323] From <ipython-input-8-49c22bec9490>:1: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [9]:
# Python optimisation variables
learning_rate = 0.5
epochs = 10
batch_size = 100

In [10]:
# Here we are choosing GradientDescentOptimizer with learning rate 0.5 which will minimize the 
# cross entropy loss
optimiser = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

In [11]:
# finally setup the initialisation operator
init_op = tf.global_variables_initializer()

In [12]:
# define an accuracy assessment operation
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [13]:
# start the session
with tf.Session() as sess:
   # initialise the variables
    sess.run(init_op)
    # how many total batches will be there
    total_batch = int(len(mnist.train.labels) / batch_size)
    for epoch in range(epochs):
        avg_cost = 0
        for i in range(total_batch):
            # It is generating next batch with 100 training examples
            batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)
            # we are feeding x as batch_x and y as batch_y
            # sess.run returns 2 values one for optimiser and another for cross_entropy for each batch
            # no need to take values for optimiser(not required)
            # return the value to c
            _, c = sess.run([optimiser, cross_entropy], 
                         feed_dict={x: batch_x, y: batch_y})
            avg_cost += c / total_batch
        print("Epoch:", (epoch + 1), "cost =", "{:.3f}".format(avg_cost))
    print(sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels}))

Epoch: 1 cost = 0.349
Epoch: 2 cost = 0.135
Epoch: 3 cost = 0.093
Epoch: 4 cost = 0.071
Epoch: 5 cost = 0.057
Epoch: 6 cost = 0.046
Epoch: 7 cost = 0.039
Epoch: 8 cost = 0.032
Epoch: 9 cost = 0.027
Epoch: 10 cost = 0.022
0.9784


In [ ]:
# only 10 epoches we are geting 97.84 accuracy in test data 